Create a Linked-Server for Active Directory Queries

- Server must be running on the domain
- Server must be running with credentials that can read the domain

In [ ]:
USE [master]
GO

/****** Object:  LinkedServer [ADSI]    Script Date: 06/12/2018 11:45:38 AM ******/
EXEC master.dbo.sp_addlinkedserver @server = N'ADSI', @srvproduct=N'Active Directory Service Interfaces', @provider=N'ADSDSOObject', @datasrc=N'adsdatasource'
 /* For security reasons the linked server remote logins password is changed with ######## */
EXEC master.dbo.sp_addlinkedsrvlogin @rmtsrvname=N'ADSI',@useself=N'True',@locallogin=NULL,@rmtuser=NULL,@rmtpassword=NULL
GO

EXEC master.dbo.sp_serveroption @server=N'ADSI', @optname=N'collation compatible', @optvalue=N'false'
GO

EXEC master.dbo.sp_serveroption @server=N'ADSI', @optname=N'data access', @optvalue=N'true'
GO

EXEC master.dbo.sp_serveroption @server=N'ADSI', @optname=N'dist', @optvalue=N'false'
GO

EXEC master.dbo.sp_serveroption @server=N'ADSI', @optname=N'pub', @optvalue=N'false'
GO

EXEC master.dbo.sp_serveroption @server=N'ADSI', @optname=N'rpc', @optvalue=N'false'
GO

EXEC master.dbo.sp_serveroption @server=N'ADSI', @optname=N'rpc out', @optvalue=N'false'
GO

EXEC master.dbo.sp_serveroption @server=N'ADSI', @optname=N'sub', @optvalue=N'false'
GO

EXEC master.dbo.sp_serveroption @server=N'ADSI', @optname=N'connect timeout', @optvalue=N'0'
GO

EXEC master.dbo.sp_serveroption @server=N'ADSI', @optname=N'collation name', @optvalue=NULL
GO

EXEC master.dbo.sp_serveroption @server=N'ADSI', @optname=N'lazy schema validation', @optvalue=N'false'
GO

EXEC master.dbo.sp_serveroption @server=N'ADSI', @optname=N'query timeout', @optvalue=N'0'
GO

EXEC master.dbo.sp_serveroption @server=N'ADSI', @optname=N'use remote collation', @optvalue=N'true'
GO

EXEC master.dbo.sp_serveroption @server=N'ADSI', @optname=N'remote proc transaction promotion', @optvalue=N'true'
GO

Query the Linked-Server

In [ ]:
DECLARE @group NVARCHAR(128) = 'AD Security Group'
DECLARE @DC1 NVARCHAR(128) = 'Your Domain - so google'
DECLARE @DC2 NVARCHAR(128) = 'Your top level domain - so com'

DECLARE @SQL NVARCHAR(MAX)
DECLARE @group_dn NVARCHAR(512)
DECLARE @result TABLE(name NVARCHAR(512))

--Get the actual group name
SET @SQL = 
'SELECT distinguishedName
FROM OPENQUERY
(ADSI,''SELECT cn, distinguishedName, dc
FROM ''''LDAP://' + @DC1 + '.' + @DC2 + '/DC=' + @DC1 + ',DC=' + @DC2 + '''''
WHERE objectCategory = ''''group'''' AND cn = ''''' + @group + ''''''')'

--PRINT @SQL
INSERT @result(name)
EXEC sp_executesql @SQL
SELECT @group_dn = name FROM @result


--https://ldapwiki.com/wiki/1.2.840.113556.1.4.1941
SET @SQL =
'SELECT *
FROM OPENQUERY (ADSI, ''<LDAP://' + @DC1 + '.' + @DC2 + '>;
(&(objectCategory=person)(memberOf:1.2.840.113556.1.4.1941:=' + @group_dn + '));
cn, sAMAccountName,mail;subtree'')
ORDER BY cn;'

--PRINT @SQL
EXEC sp_executesql @SQL